In [1]:
import pandas as pd
import json

In [2]:

#%% Opciones para visualizar data frames en consola

pd.set_option("display.max_rows",5000)
pd.set_option("display.max_columns",500)
pd.set_option("display.width",1000)

# Ingreso de datos de entrada: leer archivo JSON

archivo = "archivo_tradeview_0.json"

def f_datosent(P0_archivo="archivo_tradeview_0.json"):
    
    """
    param P0_archivio: Nombre del archivo con ubicación en la misma carpeta.
    """
    
    P1_tipo = P0_archivo.split(".")
    P1_tipo = P1_tipo[1]

    if P1_tipo=="json":
        with open(archivo, encoding="utf-8-sig") as json_file:
            datos_2 = json.load(json_file)["data"]
            
        df_datos = pd.DataFrame(datos_2["closedTransactions"]["list"])
        # Resetear index para tener todos los números de índices completos
        df_datos = df_datos.drop("info",axis=1).dropna().reset_index().drop("index",axis=1)
        
        # Tomar deseados
        df_datos = df_datos[(df_datos["type"]=="buy") | (df_datos["type"]=="sell") | (df_datos["type"]=="s/l") | (df_datos["type"]=="t/p")]
        
        # Renombrar
        
        df_datos.rename(columns  = {"SL":"sl","TP":"tp","price":"openPrice","price2":"closePrice","item":"instrument"},inplace=True) # Con inplace se guarda 
        
        return df_datos

    elif P1_tipo=="xlsx":
        
    
        df_datos = pd.read_excel(archivo)
        
        return df_datos
    
    elif P1_tipo=="csv":
        
        df_datos = pd.read_excel(archivo)
        
        return df_datos
    else:
        
        print("No manejamos tu tipo de extensión")
         
        
    
    
df_datos = f_datosent(P0_archivo=archivo)

In [4]:
df_datos.head()

,sl,tp,closeTime,commission,instrument,openTime,openPrice,closePrice,profit,size,swap,taxes,ticket,type
0,19.06755,19.46175,2019.06.18 16:40:58,0.00,usdmxn,2019.06.17 18:33:41,19.20533,19.06754,-7.23,0.01,-0.25,0.00,21988572,buy
1,108.100,109.625,2019.06.19 21:02:18,0.00,usdjpy,2019.06.17 18:34:45,108.620,108.248,-3.44,0.01,0.08,0.00,21988579,buy
2,108.049,109.615,2019.06.18 16:45:10,0.00,usdjpy,2019.06.18 05:16:26,108.379,108.514,3.73,0.03,0.00,0.00,22004255,buy
3,1.33500,1.34300,2019.06.19 15:30:06,0.00,usdcad,2019.06.18 23:03:00,1.33822,1.33499,-2.42,0.01,0.00,0.00,22053097,buy
4,1.33500,1.35000,2019.06.19 15:30:05,0.00,usdcad,2019.06.18 23:03:01,1.33822,1.33499,-2.42,0.01,0.00,0.00,22053098,buy


In [5]:
tp = df_datos.loc[:,'tp']
sl = df_datos.loc[:,'sl']
o_price = df_datos.loc[:,'openPrice']

In [6]:
o_price

0     19.20533
1      108.620
2      108.379
3      1.33822
4      1.33822
5      1346.37
6      1346.37
7      1.11909
8      1.13304
9      1.13250
10     1.31933
11     1.13637
12     1.13621
13     1.13286
14     1.13283
15     1.13272
16     1.13269
17     1.13288
18     1.13341
19     1.13310
20     1.13326
21     1.13510
22     1.12862
23     1.61682
24     1.61659
25     1.81154
26     0.69763
27     1.31317
28     1.30597
29     0.87234
30     0.86846
31     1.30788
32    19.01507
33     1404.42
34     0.91245
35     1.12053
36     1.24528
37     1.12729
38     1.30571
39    19.11317
40    19.09311
41     1.12550
42     1406.13
43     1410.68
44     1.30632
45     1.30389
46    18.97139
47     107.926
48     0.70366
49     1.12144
50     1.12144
51     0.98724
52     1.67134
53     0.70386
54    18.98049
55     1.25429
56     0.70678
57     1.12242
58     1.25081
59     1.12188
60     1.24749
61     1417.87
62     1421.22
Name: openPrice, dtype: object